## ทำดิกชันนารีเพื่อเก็บข้อมูลพิกัดและความสูงของแต่ละเรดาร์
* ข้อมูลพิกัดสถานี ความสูงจากระดับน้ำทะเล มุมยกแรก จะถูกนำไปใช้ในกระบวนการต่อไปได้

In [1]:
'''
2024.09.22
โค้ดนี้พัฒนาโดย รองศาสตราจารย์ ดร. นัฐพล มหาวิค ภาควิชาทรัพยากรธรรมชาติและสิ่งแวดล้อม คณะเกษตรศาสตร์ฯ มหาวิทยาลัยนเรศวร 
ในงานวิจัย เรื่อง "การวิจัยและพัฒนาผลิตภัณฑ์โมเสคฝนประมาณค่าจากเรดาร์ตรวจอากาศในพื้นที่ระดับลุ่มน้ำของประเทศไทยด้วยเทคโนโลยีภูมิสารสนเทศรหัสเปิด"
สนับสนุนทุนวิจัยโดยสํานักงานการวิจัยแห่งชาติ (วช.)  แผนงานการวิจัยและนวัตกรรมแผนงานด้านการบริหารจัดการภัยพิบัติทางธรรมชาติ 
ประจำปีงบประมาณ 2566  ตามสัญญา เลขที่ N25A660467 ผู้นำโค้ดนี้ไปใช้หรือดัดแปลงควรอ้างอิงงานวิจัยชิ้นนี้ตามหลักเกณฑ์การอ้างอิงสากล
เรียนหลักการเรดาร์และภูมิสารสนเทศ ที่ https://www.youtube.com/@Nattapon_Mahavik/playlists
หนังสือเรดาร์ตรวจอากาศทางอุตุนิยมวิทยา สำนักพิมพ์จุฬาฯ : https://www.chulabook.com/education/144567
หนังสือออนไลน์เรดาร์ตรวจอากาศทางอุตุนิยมวิทยา สำนักพิมพ์จุฬาฯ : https://www.chulabook.com/education/205129
ติดต่อ nattaponm@nu.ac.th
'''
import os
import pyart

def create_simplified_radar_dictionary(base_folder_path):
    radars = {}
    
    radar_folders = [f for f in os.listdir(base_folder_path) if os.path.isdir(os.path.join(base_folder_path, f))]
    
    for radar_folder in radar_folders:
        folder_path = os.path.join(base_folder_path, radar_folder)
        
        radar_files = [f for f in os.listdir(folder_path) if f.endswith('.uf.bz2') or f.endswith('.uf')]
        if not radar_files:
            print(f"No .uf.bz2 or .uf files found for {radar_folder}")
            continue
        
        file_path = os.path.join(folder_path, radar_files[0])
        
        # Handle NRT files
        if radar_folder == 'NRT' and file_path.endswith('.bz2'):
            new_file_path = file_path[:-4]  # Remove .bz2 extension
            os.rename(file_path, new_file_path)
            file_path = new_file_path
        
        try:
            radar = pyart.io.read(file_path)
            
            lon = round(radar.longitude['data'][0], 6)
            lat = round(radar.latitude['data'][0], 6)
            alt = round(radar.altitude['data'][0], 1)
            
            # Extract elevation angle from the first sweep
            el = round(radar.elevation['data'][0], 2)
            
            radars[radar_folder] = {"coords": (lon, lat, alt), "el": el}
            print(f"Successfully processed {radar_folder}")
            
        except Exception as e:
            print(f"Error processing {radar_folder}: {str(e)}")
        
        # Rename NRT file back to original name if it was changed
        if radar_folder == 'NRT' and not file_path.endswith('.bz2'):
            os.rename(file_path, file_path + '.bz2')
    
    return radars

# Main execution
base_folder_path = '../1data/0radar/1Mosaic_hourly_1hour_sotntihn/' # เปลี่ยนตรงนี้
radar_dict = create_simplified_radar_dictionary(base_folder_path)

print("\nFinal Radar Dictionary:")
print("radars = {")
for radar, data in radar_dict.items():
    print(f"    \"{radar}\": {data},")
print("}")

# Save the dictionary to a Python file
with open('./0Zprocessing_data/radar_dictionary.py', 'w') as f:
    f.write("radars = {\n")
    for radar, data in radar_dict.items():
        f.write(f"    \"{radar}\": {data},\n")
    f.write("}\n")
print("\nRadar dictionary saved to 'radar_dictionary.py'")


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119

Successfully processed CHN
Successfully processed CMP
Successfully processed CRI
Successfully processed KKN
Successfully processed KRB
Successfully processed LMP
Successfully processed NRT
Successfully processed PHS
No .uf.bz2 or .uf files found for PKT
Successfully processed SNK
Successfully processed STP
Successfully processed SVP

Final Radar Dictionary:
radars = {
    "CHN": {'coords': (100.191263, 15.157852, 40.0), 'el': 0.5},
    "CMP": {'coords': (99.188203, 10.493099, 32.0), 'el': 0.0},
    "CRI": {'coords': (99.881593, 19.961471, 444.0), '